# Module 2: LangGraph Fundamentals - HR Agent Labs (LangChain 1.0)

**Updated for LangChain 1.0 Alpha**

**Building on Module 1**: In Module 1, you built three HR agents. Now we'll learn LangGraph - the foundation that powers those agents.

**Why learn LangGraph?** Understanding LangGraph gives you:
- Full control over agent workflows
- Ability to create custom patterns
- Better debugging and optimization
- Foundation for advanced agentic systems

**Time:** 2-3 hours

## Setup: Install Dependencies

In [ ]:
# Install required packages for LangChain 1.0
!pip install --pre -U langchain langchain-openai langgraph

## Setup: Configure OpenAI API Key

In [ ]:
import os

# Set your OpenAI API key
# Option 1: For Google Colab
# from google.colab import userdata
# OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
# os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

# Option 2: Set directly (not recommended for production)
os.environ['OPENAI_API_KEY'] = 'your-api-key-here'

print("✅ API Key configured!")

---
# Lab 1: Your First LangGraph - Linear Workflow

**Objective:** Build a simple linear workflow using LangGraph's core concepts: State, Nodes, Edges, START, and END.

**Scenario:** Create an employee onboarding workflow that processes new hires step by step.

**Key Concepts:**
- **State**: Data that flows through the graph
- **Nodes**: Functions that do work
- **Edges**: Connections between nodes
- **START**: Entry point
- **END**: Exit point

## Imports

In [ ]:
from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_openai import ChatOpenAI

# Initialize LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

print("✅ Imports successful!")

## Part 1: Understanding State

**State** is the data structure that flows through your graph. It's like a shared clipboard that every node can read and update.

In [ ]:
class OnboardingState(TypedDict):
    """State for employee onboarding workflow"""
    employee_name: str
    employee_id: str
    department: str
    messages: Annotated[list, add_messages]
    validation_status: str
    tasks_completed: list

print("✅ OnboardingState defined!")
print("\nState fields:")
print("  - employee_name: Employee's full name")
print("  - employee_id: Unique employee ID")
print("  - department: Department name")
print("  - messages: Conversation history")
print("  - validation_status: Validation result")
print("  - tasks_completed: List of completed tasks")

## Part 2: Creating Nodes

**Nodes** are functions that do work. Each node receives state, processes it, and returns updates.

In [ ]:
def validate_employee(state: OnboardingState):
    """Node 1: Validate employee information"""
    print(f"📋 Step 1: Validating employee {state['employee_name']}...")
    
    # Simple validation
    if state['employee_name'] and state['employee_id']:
        return {
            "validation_status": "✅ Validated",
            "messages": [("assistant", f"Employee {state['employee_name']} (ID: {state['employee_id']}) validated successfully")]
        }
    else:
        return {
            "validation_status": "❌ Failed",
            "messages": [("assistant", "Validation failed: Missing information")]
        }

def assign_equipment(state: OnboardingState):
    """Node 2: Assign equipment based on department"""
    print(f"💻 Step 2: Assigning equipment for {state['department']} department...")
    
    equipment = {
        "Engineering": ["Laptop (MacBook Pro)", "Monitor", "Keyboard", "Mouse"],
        "HR": ["Laptop (ThinkPad)", "Headset"],
        "Sales": ["Laptop (MacBook Air)", "Phone", "CRM Access"],
        "Marketing": ["Laptop (MacBook Pro)", "Design Software License"]
    }
    
    dept_equipment = equipment.get(state['department'], ["Standard Laptop"])
    
    return {
        "tasks_completed": state['tasks_completed'] + ["Equipment assigned"],
        "messages": [("assistant", f"Assigned equipment: {', '.join(dept_equipment)}")]
    }

def setup_accounts(state: OnboardingState):
    """Node 3: Set up employee accounts"""
    print(f"🔐 Step 3: Setting up accounts for {state['employee_name']}...")
    
    accounts = ["Email", "Slack", "HR Portal", "Leave Management System"]
    
    return {
        "tasks_completed": state['tasks_completed'] + ["Accounts created"],
        "messages": [("assistant", f"Created accounts: {', '.join(accounts)}")]
    }

def send_welcome_email(state: OnboardingState):
    """Node 4: Send welcome email"""
    print(f"📧 Step 4: Sending welcome email...")
    
    return {
        "tasks_completed": state['tasks_completed'] + ["Welcome email sent"],
        "messages": [("assistant", f"Welcome email sent to {state['employee_name']}@company.com")]
    }

print("✅ All nodes defined!")

## Part 3: Building the Graph

Now we connect everything using **Edges**.

In [ ]:
# Create the graph
workflow = StateGraph(OnboardingState)

# Add nodes
workflow.add_node("validate", validate_employee)
workflow.add_node("assign_equipment", assign_equipment)
workflow.add_node("setup_accounts", setup_accounts)
workflow.add_node("send_welcome", send_welcome_email)

# Add edges (define the flow)
workflow.add_edge(START, "validate")  # START → validate
workflow.add_edge("validate", "assign_equipment")  # validate → assign_equipment
workflow.add_edge("assign_equipment", "setup_accounts")  # assign_equipment → setup_accounts
workflow.add_edge("setup_accounts", "send_welcome")  # setup_accounts → send_welcome
workflow.add_edge("send_welcome", END)  # send_welcome → END

# Compile the graph
onboarding_app = workflow.compile()

print("✅ Onboarding workflow created!")
print("\nWorkflow: START → validate → assign_equipment → setup_accounts → send_welcome → END")

## Part 4: Visualize the Graph (Optional)

In [ ]:
# Visualize the workflow
from IPython.display import Image, display

try:
    display(Image(onboarding_app.get_graph().draw_mermaid_png()))
except Exception as e:
    print(f"Visualization not available: {e}")
    print("But the workflow works perfectly!")

## Part 5: Run the Workflow

In [ ]:
# Test the workflow
initial_state = {
    "employee_name": "Karan Singh",
    "employee_id": "106",
    "department": "Engineering",
    "messages": [],
    "validation_status": "",
    "tasks_completed": []
}

print("="*70)
print("STARTING ONBOARDING WORKFLOW")
print("="*70)

result = onboarding_app.invoke(initial_state)

print("\n" + "="*70)
print("ONBOARDING COMPLETE!")
print("="*70)
print(f"\nEmployee: {result['employee_name']}")
print(f"Status: {result['validation_status']}")
print(f"Tasks completed: {len(result['tasks_completed'])}")
print(f"\nAll steps:")
for task in result['tasks_completed']:
    print(f"  ✓ {task}")

## 🎯 Exercise 1.1: Add an Orientation Node

Add a new node `schedule_orientation` that:
- Comes after `send_welcome` and before `END`
- Adds "Orientation scheduled" to tasks_completed
- Prints orientation date (e.g., "Orientation scheduled for [next Monday]")

In [ ]:
# Your code here
def schedule_orientation(state: OnboardingState):
    """TODO: Implement this node"""
    pass

# TODO: Create new workflow with the orientation node
# TODO: Test it

---
# Lab 2: Using LangChain Agents - Simple Approach

**Objective:** Learn the simple way to create agents using LangChain 1.0.

**Key Concept:** Use `create_agent` with `system_prompt` for simple cases.

## Part 1: Create a Simple Tool

In [ ]:
from langchain_core.tools import tool

# Employee database
leave_balances = {
    "101": 12,
    "102": 8,
    "103": 15,
    "104": 10,
    "105": 5
}

@tool
def check_leave_balance(employee_id: str) -> str:
    """Check the leave balance for an employee.
    
    Args:
        employee_id: The employee's ID number as a string
    
    Returns:
        A message with the employee's leave balance
    """
    balance = leave_balances.get(employee_id, 0)
    return f"Employee {employee_id} has {balance} days of leave available."

print("✅ Tool created!")

## Part 2: Create Agent with Simple System Prompt (LangChain 1.0 Way)

In [ ]:
from langchain.agents import create_agent

# Create agent with system_prompt (NEW in LangChain 1.0)
hr_agent = create_agent(
    model=llm,
    tools=[check_leave_balance],
    system_prompt="""You are a helpful HR assistant. 
    When users ask about leave balance, use the check_leave_balance tool.
    Be friendly and professional."""
)

print("✅ HR Agent created with system_prompt!")

## Part 3: Use the Agent

In [ ]:
# Human messages go in the invoke
response = hr_agent.invoke({
    "messages": [
        {"role": "user", "content": "What's my leave balance? My ID is 101"}
    ]
})

# Print the response
print(response["messages"][-1].content)

## Part 4: Try Multiple Queries

In [ ]:
# Test different queries
queries = [
    "Check leave balance for employee 102",
    "How many days does employee 103 have?",
    "Tell me about employee 105's leave"
]

for query in queries:
    print(f"\n❓ Query: {query}")
    response = hr_agent.invoke({"messages": [{"role": "user", "content": query}]})
    print(f"🤖 Response: {response['messages'][-1].content}")

## 🎯 Exercise 2.1: Add More Tools

Create additional tools for the HR agent:
1. `get_employee_info(employee_id)` - Returns name and department
2. `request_leave(employee_id, days)` - Submits a leave request

Update the agent with these new tools.

In [ ]:
# Your code here
employees = {
    "101": {"name": "Priya Sharma", "department": "Engineering"},
    "102": {"name": "Rahul Verma", "department": "Engineering"},
    "103": {"name": "Anjali Patel", "department": "HR"},
}

@tool
def get_employee_info(employee_id: str) -> str:
    """TODO: Implement this"""
    pass

# TODO: Create request_leave tool
# TODO: Create new agent with all three tools

---
## Summary

### Key Takeaways:

1. **LangChain 1.0 Changes:**
   - Use `system_prompt` instead of `prompt` parameter
   - System prompt = Agent's role and behavior (string)
   - Human messages = User queries (passed in invoke)

2. **Simple Pattern:**
   ```python
   agent = create_agent(
       model=llm,
       tools=[...],
       system_prompt="You are a helpful assistant..."
   )
   
   response = agent.invoke({
       "messages": [{"role": "user", "content": "..."}]
   })
   ```

3. **When You Need More:**
   - For dynamic prompts based on state → Use prompt functions
   - For complex workflows → Use LangGraph StateGraph
   - For simple agents → Use create_agent with system_prompt

### Next Steps:
- Complete the exercises
- Experiment with different system prompts
- Try creating your own tools
- Build a complete HR workflow combining LangGraph + Agents